The aim of this notebook is to help user understand his/her MPESA transactions in a given time frame. What is required is an pdf upload of the mpesa statement and supplied password.

In [1]:
#install packages/modules if missing in your local machine
#pip install --upgrade plotly
#pip install streamlit

In [2]:
# Supress unnecessary warnings so that presentation looks clean
import warnings
warnings.filterwarnings('ignore')

#modules/packages required
import os

#for data manipulation/wrangling
import numpy as np
from numpy import int64
import pandas as pd

#for data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px

#for date manipulation
import datetime as datetime
import calendar


#for pdf extraction as pdf
import tabula


# to print out all the outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

In [3]:
# Function to calculate missing values by column
def missing_values_table(df):
    #Total missing values 
    mis_val = df.isnull().sum()
    
    #percentage of missing values
    mis_val_percent = 100 * df.isnull().sum() / len(df)
    
    # Make a table with the results
    mis_val_table = pd.concat([mis_val,mis_val_percent], axis =1)
    
    # Rename the colums
    mis_val_table_ren_columns = mis_val_table.rename(
    columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    
    # Sort the table by percentage of missing descending
    mis_val_table_ren_columns = mis_val_table_ren_columns[
        mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
    
    # Print some summary information
    print("Your selected dataframe has " + str(df.shape[1])+ " columns.\n"
         "There are " + str(mis_val_table_ren_columns.shape[0])+
         " columns that have missing values.")
    
    # Return the dataframe with missing information
    return mis_val_table_ren_columns

#function for checking missing values per column

#Create a new function:
def num_missing(x):
    return sum(x.isnull())


Data uploaded is done here. Requires the pdf statement and password

In [4]:
dfs = tabula.read_pdf('../data/raw_data/MPESA_Statement_20191221_to_20201221_254711170904.pdf',pages="all",multiple_tables=True,password = '27791676',stream=True, lattice=  True)

In [5]:
#check the number of tables
print("There are {0} tables in the data".format(len(dfs)))

There are 34 tables in the data


In [6]:
df = dfs[1]
df.head()

,0,1,2,3,4,5,6
0,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance
1,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,NaN,-28.00,285.72
2,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi\rmobiles ventures ltd Roysambu near Pemca Holdings\rLumumba Drive.,Completed,NaN,"-1,200.00",313.72
3,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc.\r0711170904,Completed,NaN,-500.00,"1,513.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"2,013.72"


In [7]:
df_s = dfs[3]
#grab the first row for the header
new_header = df_s.iloc[0] 
#take the data less the header row
df_s = df_s[1:]
df_s.columns = new_header #set the header row as the df header
df_s.shape

(49, 8)

In [8]:
df_s = df_s.rename(columns={np.nan: 'null_column'})
df_s.head()
# df_s = df_s.loc[:, df_s.columns.notnull()]
# df_s.columns

,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance,NaN
1,OLC0HJ3QEG,2020-12-12 18:26:21,Customer Withdrawal At Agent Till 439844 - Junja\rCompany ltd Homeyard Twin electronics Agg,Completed,NaN,"-2,500.00","2,087.93",NaN
2,OLC9HDBT4P,2020-12-12 16:40:28,Customer Transfer to 254704700111 - KIPYEGON\rRONNY BORE,Completed,NaN,-177.00,"4,587.93",NaN
3,OLC6HBKFQ2,2020-12-12 16:03:48,Funds received from 0791913116 - Kipyegon Titus,Completed,"1,700.00",NaN,"4,764.93",NaN
4,OLC6H9C29C,2020-12-12 15:16:16,Customer Transfer of Funds Charge,Completed,NaN,-26.00,"3,064.93",NaN
5,OLC6H9C29C,2020-12-12 15:16:16,Customer Transfer to 254724533738 - VIOLA\rCHEPKEMOI,Completed,NaN,"-1,028.00","3,090.93",NaN


In [9]:
df_s.columns = df_s.columns.fillna('null_column')
df_s.head()

,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance,null_column
1,OLC0HJ3QEG,2020-12-12 18:26:21,Customer Withdrawal At Agent Till 439844 - Junja\rCompany ltd Homeyard Twin electronics Agg,Completed,NaN,"-2,500.00","2,087.93",NaN
2,OLC9HDBT4P,2020-12-12 16:40:28,Customer Transfer to 254704700111 - KIPYEGON\rRONNY BORE,Completed,NaN,-177.00,"4,587.93",NaN
3,OLC6HBKFQ2,2020-12-12 16:03:48,Funds received from 0791913116 - Kipyegon Titus,Completed,"1,700.00",NaN,"4,764.93",NaN
4,OLC6H9C29C,2020-12-12 15:16:16,Customer Transfer of Funds Charge,Completed,NaN,-26.00,"3,064.93",NaN
5,OLC6H9C29C,2020-12-12 15:16:16,Customer Transfer to 254724533738 - VIOLA\rCHEPKEMOI,Completed,NaN,"-1,028.00","3,090.93",NaN


In [10]:
# new_header = df.iloc[0] #grab the first row for the header
# df = df[1:] #take the data less the header row
# df.columns = new_header #set the header row as the df header
# df.shape

In [11]:
df.head()

,0,1,2,3,4,5,6
0,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance
1,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,NaN,-28.00,285.72
2,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi\rmobiles ventures ltd Roysambu near Pemca Holdings\rLumumba Drive.,Completed,NaN,"-1,200.00",313.72
3,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc.\r0711170904,Completed,NaN,-500.00,"1,513.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"2,013.72"


In [12]:
# df_new = pd.DataFrame(np.concatenate([df.values, df_s.values]), columns=df.columns)
# df_new.shape

In [13]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

no_tables = [3]
for i in range(2, len(dfs)):
    df_s = dfs[i]
    #grab the first row for the header
    new_header = df_s.iloc[0] 
    #take the data less the header row
    df_s = df_s[1:] 
    
    #set the header row as the df header
    df_s.columns = new_header
    
    print(df_s.shape)
    
    ##rename null headers
    if ((df_s.columns).isna().any() == True):
        df_s.columns = df_s.columns.fillna('null_column')
    else:
        pass
    #print(df_s.columns)
    
    
    #check missing values
    print(missing_values_table(df_s))
    
    # dropping columns with > 98% missing
    missing_df = missing_values_table(df_s);
    missing_columns = list(missing_df[missing_df['% of Total Values']> 98].index)
    print('We will remove %d columns.' % len(missing_columns))
    #print(missing_columns)
    df_s.drop(missing_columns, axis =1, inplace=True)
    
    
    df = pd.DataFrame(np.concatenate([df.values, df_s.values]), columns=df.columns)
    
    
df.shape

(48, 8)
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
             Missing Values  % of Total Values
0                                             
null_column  48              100.0            
Paid In      35              72.9             
Withdrawn    13              27.1             
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
We will remove 1 columns.
(49, 8)
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
             Missing Values  % of Total Values
0                                             
null_column  49              100.0            
Paid In      40              81.6             
Withdrawn    9               18.4             
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
We will remove 1 columns.
(48, 8)
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
             Missing Values  

There are 3 columns that have missing values.
We will remove 1 columns.
(55, 8)
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
             Missing Values  % of Total Values
0                                             
null_column  55              100.0            
Paid In      47              85.5             
Withdrawn    8               14.5             
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
We will remove 1 columns.
(55, 8)
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
             Missing Values  % of Total Values
0                                             
null_column  55              100.0            
Paid In      41              74.5             
Withdrawn    14              25.5             
Your selected dataframe has 8 columns.
There are 3 columns that have missing values.
We will remove 1 columns.
(48, 8)
Your selected dataframe has 8 columns.
The

(1597, 7)

In [14]:
(df_s.columns).isna().any()

False

In [15]:
df_s

,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance
1,NLN3V26FE5,2019-12-23 08:55:22,Business Payment from 329299 - STANDARD\rCHARTERED BANK via API,Completed,"10,000.00",NaN,"10,000.00"
2,NLN7V0B34X,2019-12-23 07:25:35,OverDraft of Credit Party,Completed,177.54,NaN,97.00
3,NLN7V0B34X,2019-12-23 07:25:35,Customer Transfer of Funds Charge,Completed,NaN,-97.00,0.00
4,NLN7V0B34X,2019-12-23 07:25:35,Customer Transfer Fuliza M-Pesa to 254723576355 -\rNOAH KIPTANUI,Completed,NaN,"-10,100.00",-80.54
5,NLM7UYXCVH,2019-12-22 23:54:25,Airtime Purchase,Completed,NaN,"-1,250.00","10,019.46"
6,NLM5UWEYHX,2019-12-22 21:01:19,Merchant Payment Online to 801189 - SWAN\rRESTAURANT,Completed,NaN,"-2,500.00","11,269.46"
7,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"13,769.46"
8,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer to 254724775068 - CALEB\rCHERUIYOT BIEGON,Completed,NaN,"-2,028.00","13,810.46"
9,NLL1TEOQH1,2019-12-21 08:05:26,Withdrawal Charge,Completed,NaN,-162.00,"15,838.46"
10,NLL1TEOQH1,2019-12-21 08:05:26,Customer Withdrawal At Agent Till 233409 - Natsars Ltd\rZimmerman Area Behind Co-op Bank,Completed,NaN,"-15,000.00","16,000.46"


In [16]:
range(2, len(dfs))

range(2, 34)

In [17]:
df.head()

,Receipt No.,Completion Time,Details,Transaction\rStatus,Paid In,Withdrawn,Balance
0,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,NaN,-28.00,285.72
1,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi\rmobiles ventures ltd Roysambu near Pemca Holdings\rLumumba Drive.,Completed,NaN,"-1,200.00",313.72
2,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc.\r0711170904,Completed,NaN,-500.00,"1,513.72"
3,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"2,013.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer to 254711361054 - PATRICK MUUO\rKINGOO,Completed,NaN,"-1,528.00","2,054.72"


In [18]:
print(df.columns)
df.rename(columns = {'Receipt No.':'receipt_no','Completion Time':'completion_time',
                          'Details':'details','Transaction\rStatus':'status', 'Paid In':'paid_in', 'Withdrawn':'withdrawn', 'Balance':'balance'}, inplace = True) 

Index(['Receipt No.', 'Completion Time', 'Details', 'Transaction\rStatus',
       'Paid In', 'Withdrawn', 'Balance'],
      dtype='object', name=0)


In [19]:
df.head()

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance
0,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,NaN,-28.00,285.72
1,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi\rmobiles ventures ltd Roysambu near Pemca Holdings\rLumumba Drive.,Completed,NaN,"-1,200.00",313.72
2,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc.\r0711170904,Completed,NaN,-500.00,"1,513.72"
3,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"2,013.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer to 254711361054 - PATRICK MUUO\rKINGOO,Completed,NaN,"-1,528.00","2,054.72"


In [20]:
#drop row with null receipt number
#mpesa_df = mpesa.dropna(subset = ['Receipt No.'], how='all', inplace=True)
mpesa_df = df[df['receipt_no'].notna()]

In [21]:
#clean the text columns
mpesa_df['details'] = mpesa_df['details'].str.replace('\r',' ')

In [22]:
print(mpesa_df.shape)
mpesa_df.head()

(1597, 7)


,receipt_no,completion_time,details,status,paid_in,withdrawn,balance
0,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,NaN,-28.00,285.72
1,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi mobiles ventures ltd Roysambu near Pemca Holdings Lumumba Drive.,Completed,NaN,"-1,200.00",313.72
2,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc. 0711170904,Completed,NaN,-500.00,"1,513.72"
3,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,NaN,-41.00,"2,013.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer to 254711361054 - PATRICK MUUO KINGOO,Completed,NaN,"-1,528.00","2,054.72"


In [23]:
missing_values_table(mpesa_df)

Your selected dataframe has 7 columns.
There are 2 columns that have missing values.


,Missing Values,% of Total Values
0,,
paid_in,1295,81.1
withdrawn,302,18.9


In [24]:
#filling null values in paid_in and withdrawn columns
mpesa_df['paid_in'] = mpesa_df['paid_in'].fillna(0)
mpesa_df['withdrawn'] = mpesa_df['withdrawn'].fillna(0)
mpesa_df['balance'] = mpesa_df['balance'].fillna(0)

In [25]:
mpesa_df.head()

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance
0,OLL9SJFI4Z,2020-12-21 16:27:39,Withdrawal Charge,Completed,0,-28.00,285.72
1,OLL9SJFI4Z,2020-12-21 16:27:39,Customer Withdrawal At Agent Till 197334 - Malezi mobiles ventures ltd Roysambu near Pemca Holdings Lumumba Drive.,Completed,0,"-1,200.00",313.72
2,OLL6SCJSFO,2020-12-21 13:56:16,Pay Bill Online to 200200 - Safaricom Post Paid Acc. 0711170904,Completed,0,-500.00,"1,513.72"
3,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer of Funds Charge,Completed,0,-41.00,"2,013.72"
4,OLL9S703PL,2020-12-21 11:53:08,Customer Transfer to 254711361054 - PATRICK MUUO KINGOO,Completed,0,"-1,528.00","2,054.72"


In [26]:
mpesa_df.to_csv("../data/notebook_outputs/clean_mpesa_transactions.csv", index = False)

In [27]:
mpesa_df.dtypes

0
receipt_no         object
completion_time    object
details            object
status             object
paid_in            object
withdrawn          object
balance            object
dtype: object

In [28]:
#cleaning the numerical columns
num_col = ['paid_in','withdrawn','balance']
for col in num_col:
    mpesa_df[col] = mpesa_df[col].replace(',', '',regex=True)
    mpesa_df[col] = pd.to_numeric(mpesa_df[col])
    
mpesa_df['completion_time']= pd.to_datetime(mpesa_df['completion_time'])
mpesa_df['details'] = mpesa_df['details'].astype(str)

In [46]:
#extract month transaction
mpesa_df['year'] = mpesa_df['completion_time'].dt.year
mpesa_df['month'] = mpesa_df['completion_time'].dt.month
mpesa_df['month'] = mpesa_df['month'].apply(lambda x: calendar.month_name[x])

mpesa_df['transactions_cohort']= mpesa_df['month'].astype(str) + "." + mpesa_df['year'].astype(str)

In [47]:
#sorting df by date
mpesa_df=mpesa_df.sort_values(by=['completion_time'],ascending =True)
mpesa_df.head()

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance,year,month,transactions.group,transactions_cohort
1596,NLL6TDOVMM,2019-12-21 07:07:14,Airtime Purchase,Completed,0.0,100.0,31000.46,2019,December,airtime,December.2019
1595,NLL1TEOQH1,2019-12-21 08:05:26,Customer Withdrawal At Agent Till 233409 - Natsars Ltd Zimmerman Area Behind Co-op Bank,Completed,0.0,15000.0,16000.46,2019,December,withdrawals,December.2019
1594,NLL1TEOQH1,2019-12-21 08:05:26,Withdrawal Charge,Completed,0.0,162.0,15838.46,2019,December,withdrawal_charge,December.2019
1593,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer to 254724775068 - CALEB CHERUIYOT BIEGON,Completed,0.0,2028.0,13810.46,2019,December,customer_transfer,December.2019
1592,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer of Funds Charge,Completed,0.0,41.0,13769.46,2019,December,funds_charge,December.2019


In [48]:
#group sum
#mpesa_df['pay_bill_charges'] = np.where((mpesa_df['details'].str.contains('Pay Bill Charge')),'paybill', 'Non_paybill')

In [49]:
#mpesa_df['customer_transfers'] = np.where((mpesa_df['details'].str.contains('Customer Transfer to')),'customer_transfer', 'Non_customer_transfer')

In [50]:
mpesa_df['withdrawn'] = abs(mpesa_df['withdrawn'])

In [51]:
mpesa_df.head()

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance,year,month,transactions.group,transactions_cohort
1596,NLL6TDOVMM,2019-12-21 07:07:14,Airtime Purchase,Completed,0.0,100.0,31000.46,2019,December,airtime,December.2019
1595,NLL1TEOQH1,2019-12-21 08:05:26,Customer Withdrawal At Agent Till 233409 - Natsars Ltd Zimmerman Area Behind Co-op Bank,Completed,0.0,15000.0,16000.46,2019,December,withdrawals,December.2019
1594,NLL1TEOQH1,2019-12-21 08:05:26,Withdrawal Charge,Completed,0.0,162.0,15838.46,2019,December,withdrawal_charge,December.2019
1593,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer to 254724775068 - CALEB CHERUIYOT BIEGON,Completed,0.0,2028.0,13810.46,2019,December,customer_transfer,December.2019
1592,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer of Funds Charge,Completed,0.0,41.0,13769.46,2019,December,funds_charge,December.2019


In [54]:
text_group = []
for index, row in mpesa_df.iterrows():
    if 'Funds Charge' in row['details']:
        text_group.append('funds_charge')
    elif 'Business Payment from' in row['details']:
        text_group.append('business_deposit')
    elif 'Loan Repayment' in row['details']:
        text_group.append('loan_payment')
    elif 'Receive International Transfer From' in row['details']:
        text_group.append('international_deposits')
    elif 'Airtime' in row['details']:
        text_group.append('airtime')
    elif 'Customer Transfer to' in row['details']:
        text_group.append('customer_transfer')
    elif 'Customer Transfer Fuliza' in row['details']:
        text_group.append('fuliza')   
    elif 'Customer Withdrawal At' in row['details']:
        text_group.append('withdrawals')
    elif 'Withdrawal Charge' in row['details']: 
        text_group.append('withdrawal_charge')
    elif 'Buy Bundles' in row['details']: 
        text_group.append('bundles')
    elif 'Pay Bill' in row['details']:
        text_group.append('bill_payments')
    elif 'Pay Bill Charge' in row['details']:
        text_group.append('bill_charges')
    elif 'Merchant Payment' in row['details']: 
        text_group.append('merchant_payment')
    elif 'Funds received from' in row['details']: 
        text_group.append('deposit_sender')
    elif 'OverDraft' in row['details']: 
        text_group.append('overdraft')
    elif 'Promotion Payment from' in row['details']: 
        text_group.append('deposit_promotion')
    elif 'Deposit of Funds at ' in row['details']: 
        text_group.append('deposits')
    elif 'M-Shwari Deposit' in row['details']: 
        text_group.append('mshwari_deposits')
    elif 'M-Shwari Withdraw' in row['details']: 
        text_group.append('mshwari_withdrawals ')
    elif 'Pay Merchant Charge' in row['details']: 
        text_group.append('mechant_charges')
    elif 'Reversal' in row['details']: 
        text_group.append('reversal')
    elif 'M-Shwari Lock Deposit' in row['details']: 
        text_group.append('mshwari_deposit')
    elif 'M-Shwari Loan Disburse' in row['details']: 
        text_group.append('mshwari_loan')
    else :
        text_group.append('error')
        
mpesa_df['transactions.group'] = text_group
        

In [55]:
mpesa_df[mpesa_df['transactions.group']=='error']

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance,year,month,transactions.group,transactions_cohort


In [56]:
mpesa_df.head()

,receipt_no,completion_time,details,status,paid_in,withdrawn,balance,year,month,transactions.group,transactions_cohort
1596,NLL6TDOVMM,2019-12-21 07:07:14,Airtime Purchase,Completed,0.0,100.0,31000.46,2019,December,airtime,December.2019
1595,NLL1TEOQH1,2019-12-21 08:05:26,Customer Withdrawal At Agent Till 233409 - Natsars Ltd Zimmerman Area Behind Co-op Bank,Completed,0.0,15000.0,16000.46,2019,December,withdrawals,December.2019
1594,NLL1TEOQH1,2019-12-21 08:05:26,Withdrawal Charge,Completed,0.0,162.0,15838.46,2019,December,withdrawal_charge,December.2019
1593,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer to 254724775068 - CALEB CHERUIYOT BIEGON,Completed,0.0,2028.0,13810.46,2019,December,customer_transfer,December.2019
1592,NLL7U6LEUV,2019-12-21 22:19:36,Customer Transfer of Funds Charge,Completed,0.0,41.0,13769.46,2019,December,funds_charge,December.2019


In [57]:
#undersanding the performance of various transactions groups over time
table_withdrawals = pd.pivot_table(mpesa_df,index=['transactions.group'],columns = ['transactions_cohort'],
                       values = ['withdrawn'],aggfunc={'withdrawn':np.sum},
                      margins=True,margins_name='Grand Total').reset_index()


table_withdrawals.columns = [' '.join(col).strip() for col in table_withdrawals.columns.values]
table_withdrawals = table_withdrawals.sort_values('transactions.group', ascending=False)
table_withdrawals.columns = table_withdrawals.columns.str.replace("withdrawn ", "")
table_withdrawals = table_withdrawals[table_withdrawals['Grand Total'] !=0].fillna(0)

# #columns ordering
#table_withdrawals = table_withdrawals[['transactions.group','March', 'April', 'May', 'June', 'July', 'August','Grand Total']]

table_withdrawals

,transactions.group,April.2020,August.2020,December.2019,December.2020,February.2020,January.2020,July.2020,June.2020,March.2020,May.2020,November.2020,October.2020,September.2020,Grand Total
21,withdrawals,30350.00,28200.00,25800.00,20600.00,18700.00,41350.00,27850.00,2500.00,6650.00,18500.00,62700.00,46500.00,10700.00,340400.00
20,withdrawal_charge,302.00,446.00,324.00,308.00,301.00,467.00,367.00,55.00,112.00,296.00,683.00,587.00,162.00,4410.00
19,reversal,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,300.00,0.00,300.00
15,mshwari_deposits,25000.00,0.00,0.00,0.00,0.00,0.00,18060.00,0.00,0.00,2000.00,38000.00,51200.00,5000.00,139260.00
14,mshwari_deposit,0.00,0.00,0.00,0.00,0.00,0.00,0.00,3000.00,0.00,0.00,0.00,0.00,0.00,3000.00
13,merchant_payment,12737.00,33912.00,2500.00,15200.00,7031.00,7607.00,52030.00,13578.00,322.00,9328.00,30048.00,15951.00,15784.00,216028.00
12,mechant_charges,0.00,0.00,0.00,0.00,0.00,0.00,0.00,10.00,0.00,0.00,0.00,0.00,0.00,10.00
11,loan_payment,3873.04,216.38,3483.31,4664.95,22168.69,13575.43,2001.23,1954.41,15226.92,1147.51,132.91,1081.21,2295.75,71821.74
9,funds_charge,490.00,364.00,502.00,359.00,935.00,734.00,564.00,384.00,703.00,230.00,598.00,522.00,390.00,6775.00
8,fuliza,300.00,330.00,13350.00,1270.00,12865.00,8044.00,0.00,3028.00,6590.00,0.00,0.00,500.00,5280.00,51557.00


In [59]:
#undersanding the performance of various transactions groups over time
table_deposits = pd.pivot_table(mpesa_df,index=['transactions.group'],columns = ['transactions_cohort'],
                       values = ['paid_in'],aggfunc={'paid_in':np.sum},
                      margins=True,margins_name='Grand Total').reset_index()


table_deposits.columns = [' '.join(col).strip() for col in table_deposits.columns.values]
table_deposits = table_deposits.sort_values('transactions.group', ascending=False)
table_deposits.columns = table_deposits.columns.str.replace("paid_in ", "")
table_deposits = table_deposits[table_deposits['Grand Total'] !=0].fillna(0)

#columns reordering
#table_deposits = table_deposits[['transactions.group','March', 'April', 'May', 'June', 'July', 'August','Grand Total']]


table_deposits

,transactions.group,April.2020,August.2020,December.2019,December.2020,February.2020,January.2020,July.2020,June.2020,March.2020,May.2020,November.2020,October.2020,September.2020,Grand Total
19,reversal,0.00,0.00,0.00,0.00,0.00,0.00,100.00,0.00,0.00,0.00,0.00,0.00,0.0,100.0
18,overdraft,911.26,214.23,3443.86,4618.74,10663.69,7110.42,1981.41,1935.05,8688.92,1126.24,131.59,1060.59,2273.0,44159.0
17,mshwari_withdrawals,0.00,0.00,0.00,0.00,0.00,0.00,18000.00,500.00,0.00,0.00,38050.00,51200.00,20300.0,128050.0
16,mshwari_loan,0.00,5000.00,0.00,0.00,8300.00,8300.00,0.00,0.00,8300.00,0.00,0.00,0.00,0.0,29900.0
10,international_deposits,0.00,0.00,0.00,5632.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,5632.0
7,deposits,0.00,10000.00,0.00,0.00,35000.00,0.00,0.00,0.00,35000.00,0.00,0.00,0.00,10000.0,90000.0
6,deposit_sender,33050.00,4008.00,42890.00,24830.00,34120.00,46930.00,37190.00,7780.00,1210.00,12050.00,63180.00,51884.00,1000.0,360122.0
5,deposit_promotion,0.00,0.00,0.00,0.00,2050.00,4080.00,0.00,0.00,0.00,0.00,2000.00,0.00,0.0,8130.0
3,business_deposit,109500.00,77000.00,14788.00,87880.00,29750.00,73350.00,108080.00,57000.00,43800.00,60880.00,162450.00,110550.00,80000.0,1015028.0
22,Grand Total,143461.26,96222.23,61121.86,122960.74,119883.69,139770.42,165351.41,67215.05,96998.92,74056.24,265811.59,214694.59,113573.0,1681121.0


In [61]:
# Group the data frame by month and item and extract a number of stats from each group
mpesa_agg =mpesa_df.groupby(['transactions_cohort','transactions.group'], as_index= False).agg({
        # Find the min, max, and sum of the duration column
        'withdrawn': ["count", sum],
        # find the number of network type entries
        'paid_in': [sum]
    }
)

#mpesa_agg.set_index('transactions.group',inplace= True)
#mpesa_agg = mpesa_agg.reset_index()
mpesa_agg.columns = [' '.join(col).strip() for col in mpesa_agg.columns.values]
#mpesa_agg.loc['Total']= mpesa_agg.sum(numeric_only=True, axis=0)
mpesa_agg = mpesa_agg.where(pd.notnull(mpesa_agg), None)
mpesa_agg


,transactions_cohort,transactions.group,withdrawn count,withdrawn sum,paid_in sum
0,April.2020,airtime,9,785.00,0.00
1,April.2020,bill_payments,27,26954.00,0.00
2,April.2020,business_deposit,5,0.00,109500.00
3,April.2020,customer_transfer,24,41801.00,0.00
4,April.2020,deposit_sender,4,0.00,33050.00
5,April.2020,fuliza,1,300.00,0.00
6,April.2020,funds_charge,8,490.00,0.00
7,April.2020,loan_payment,3,3873.04,0.00
8,April.2020,merchant_payment,10,12737.00,0.00
9,April.2020,mshwari_deposits,1,25000.00,0.00


In [62]:
fig = px.treemap(mpesa_agg, path=['transactions_cohort', 'transactions.group'], values='withdrawn count')
#fig = px.histogram(mpesa_agg, x='withdrawn count',color = "month", title='Rating distribution')
fig.show()

In [63]:

fig =px.treemap(mpesa_agg, path=['transactions_cohort', 'transactions.group'], values='withdrawn count',
                 color='withdrawn sum',color_continuous_scale='RdBu',
                  color_continuous_midpoint=np.average(mpesa_agg['withdrawn sum'], weights=mpesa_agg['withdrawn count']))
# this is what I don't like, accessing traces like this
fig.data[0].textinfo = 'label+text+value'

#fig.layout.hovertamplate = '%{label}<br>%{value}'
fig.data[0].hovertemplate = '%{label}<br>%{value}'
fig.show()

In [43]:
fig =px.treemap(mpesa_agg, path=['month', 'transactions.group'], values='withdrawn sum')
fig.data[0].textinfo = 'label+text+value'

#fig.layout.hovermode = False
fig.data[0].hovertemplate = '%{label}<br>%{value}'
fig.show()

In [44]:
fig = px.sunburst(mpesa_agg, path=['month', 'transactions.group'], values='withdrawn count', title='Monthly usage')
#fig = px.histogram(mpesa_agg, x='withdrawn count',color = "month", title='Rating distribution')
fig.show()

In [45]:
mpesa_df.to_csv("../data/notebook_outputs/aggregated_mpesa_charges.csv")